In [89]:
import requests
import pickle
import mlflow
import pandas
from sklearn.feature_extraction import DictVectorizer

In [90]:
dv_path = './output/dv.pkl'
model_uri = './artifacts/2/bc86bb7220a14f1b94b038c8e6d52c16/artifacts/model'

Start server with mlflow models serve --model-uri ./artifacts/2/bc86bb7220a14f1b94b038c8e6d52c16/artifacts/model
This will create a conda env named mlflow-b1bd6a48ce10354a9129faed002053f1fc6bb46c (if it does not exists). Also will install the requirements from model definition (check MLmodel and, depending on the setup, conda.yaml or python_env.yaml+requirements.txt)

Use mlflow models build-docker --model-uri './artifacts/2/bc86bb7220a14f1b94b038c8e6d52c16/artifacts/model' -n 'mlflow-inference-server' to create a docker image with the model and all needed to serve (nginx, gunicorn, mlflow and so on), using python_function flavor. Takes some minutes to finish. Currently, error arises. Not fixed yet

In [91]:
def load_dv(filename):
    with open(filename,'rb') as f_in:
        return pickle.load(f_in)

In [92]:
dv = load_dv(dv_path)

In [93]:
x = dv.transform({'PU_DO': '43_151', 'trip_distance': 1.01})

In [94]:
# model = mlflow.pyfunc.load_model(model_uri)

In [95]:
model = mlflow.sklearn.load_model(model_uri)

In [96]:
%time model.predict(x)

CPU times: total: 0 ns
Wall time: 5 ms


array([5.91443593])

In [97]:
payload = pandas.DataFrame.sparse.from_spmatrix(x).to_json(orient='split')

In [100]:
%time r = requests.post('http://127.0.0.1:5000/invocations', data=payload, headers={'Content-Type': 'application/json'})

CPU times: total: 0 ns
Wall time: 2.31 s


In [101]:
r.text

'[5.91443592786821]'